# Data pre-processing & tokenization

CS685 Spring 2022 <br />
Feb. 24, 2022<br />
Hongyu Tu <br />

In [1]:
import re
import time
import jieba
import pickle
import asyncio
import datetime
import requests
import numpy as np
import pandas as pd 
import nest_asyncio
from utils import *
import jieba.analyse
from tqdm import tqdm
from os import listdir
from collections import Counter
from bilibili_api import video, sync

jieba.initialize() 
nest_asyncio.apply()
# jieba.analyse.set_stop_words('stopwords.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\TomTu\AppData\Local\Temp\jieba.cache
Loading model cost 0.599 seconds.
Prefix dict has been built successfully.


In [2]:
df = pd.read_csv('out.csv')

f = open('stopwords.txt', encoding="utf8")
stop_word = f.read().split('\n')
f.close()

In [3]:
cat_dic = init_category_dic()
tid_lst = np.array(list(cat_dic.keys()))

In [4]:
bc_lst = df['Bullet Chat'].tolist()
fq_lst = df['Frequency'].tolist()
ct_lst = df['Category ID'].tolist()

In [5]:
token_dict = {}
for idx in tqdm(range(len(bc_lst))):
    tid = ct_lst[idx]
    if not pd.isna(bc_lst[idx]) and tid in tid_lst: 
        lst_idx = np.where(tid_lst==tid)[0][0]
        
        seg_list = list(jieba.cut_for_search(bc_lst[idx]))
        if len(bc_lst[idx]) <= 5:
            seg_list.append(bc_lst[idx])
        for token in seg_list:
            if token not in stop_word and len(token) > 2 and len(set([_ for _ in token])) > 1:
                if token in token_dict:
                    tmp = np.eye(126)[lst_idx] * fq_lst[idx]
                    token_dict[token] = token_dict[token] + tmp
                else:
                    token_dict[token] = np.eye(126)[lst_idx] * fq_lst[idx]

100%|████████████████████████████████████████████████████████████████████████| 908645/908645 [02:44<00:00, 5533.95it/s]


In [6]:
token_lst = np.array(list(token_dict.keys()))
cnt_lst = np.array([sum(token_dict[i]) for i in token_lst])
dist_lst = np.array(list(token_dict.values()))/cnt_lst[:,None]
general_lst = np.array([len(np.where(i != 0)[0]) for i in dist_lst])

In [7]:
df2 = pd.DataFrame(np.array([token_lst, cnt_lst, general_lst, ]).T, \
                   columns = ['Word Token', 'Total Apperance', 'Category Apperance'])

In [8]:
df2.sort_values(by=['Category Apperance'])[-25:]

,Word Token,Total Apperance,Category Apperance
80431,这运镜,15.0,9
10323,五百年,25.0,9
14369,文科生,36.0,9
4878,Woc,14.0,9
86565,潘长江,22.0,9
18679,玩尬的是吧,15.0,9
24019,弹幕应援,14.0,9
81859,不值一提,19.0,9
6546,翘起来,11.0,9
34470,iPad,33.0,9


In [9]:
df2.sort_values(by=['Total Apperance'])[-25:]

,Word Token,Total Apperance,Category Apperance
15708,老婆老婆,97.0,22
23326,张国伟,97.0,4
14077,迪士尼,97.0,19
29050,2008,97.0,10
11326,cao,97.0,34
91526,这个我会,97.0,7
678,差不多,978.0,66
104884,五边形,98.0,4
3395,可以可以,98.0,34
8400,第一名,98.0,29
